In [1]:
import pandas as pd
from importlib import reload
import sys
sys.path.append("../../../code/")
import quantification
pd.set_option('display.max_rows', 500)

alternation_set = ["man", "werden"]
alternating = "alternating_nonagentivity_nonprimed"

In [18]:
df = pd.read_csv("../../RBC.csv", index_col=0)

## Annotation

In [19]:
import annotate_nonagentivity #excluding instructions as these were pre-formulated

In [20]:
df[alternating].fillna(value="no", inplace=True)
df.to_csv("../../RBC.csv")

In [21]:
#after annotating all non-instructions, only four interactions included more than one variant
alt = df[df.alternating_nonagentivity_nonprimed=="yes"]
multiple_variants = alt.groupby("interaction_id").size() > 1
alt[alt["interaction_id"].isin(multiple_variants[multiple_variants].index)]

,id,word,lemma,pos_finegrained,pos_coarse,speaker,interaction_id,turn_id,merged,participant_id,setting,start,end,alternating_nonagentivity_nonprimed,alternating_thanking
1737,1738,man,man,PIS.Nom.Sg.*,PIS,S,4,32,no,20170717A,01_S,183.248723,186.662689,yes,no
1741,1742,man,man,PIS.Nom.Sg.*,PIS,S,4,32,no,20170717A,01_S,183.248723,186.662689,yes,no
4605,4606,man,man,PIS.Nom.Sg.*,PIS,S,10,34,no,20170717C,01_S,187.887067,190.420894,yes,no
4705,4706,man,man,PIS.Nom.Sg.*,PIS,S,10,46,yes,20170717C,01_S,238.314052,246.664161,yes,no
11415,11416,man,man,PIS.Nom.Sg.*,PIS,S,26,8,no,20170718B,02_S,38.966657,48.116286,yes,no
11618,11619,man,man,PIS.Nom.Sg.*,PIS,S,26,28,no,20170718B,02_S,137.836476,143.945199,yes,no
21694,21695,man,man,PIS.Nom.Sg.*,PIS,S,51,22,yes,20170719C,03_F,117.66862,134.555127,yes,no
21699,21700,man,man,PIS.Nom.Sg.*,PIS,S,51,22,yes,20170719C,03_F,117.66862,134.555127,yes,no


## Overview

In [22]:
import pandas as pd
df = pd.read_csv("../../RBC.csv", index_col=0)
df.groupby("speaker").alternating_nonagentivity_nonprimed.value_counts()

speaker      alternating_nonagentivity_nonprimed
A            no                                     18766
             yes                                        2
Instruction  no                                      6363
S            no                                     14356
             yes                                       21
Name: alternating_nonagentivity_nonprimed, dtype: int64

In [23]:
len(df[df.alternating_nonagentivity_nonprimed=="yes"])

23

# Creating DataFrame for modeling

In [31]:
#reading DataFrame with column "alternating" indicating where there was an opportunity ("yes") to choose a variant from the alternation set or not ("no")
df = pd.read_csv("../../RBC.csv", index_col=0, na_filter=False, sep=",")

In [32]:
#adding information on lexical quasi-persistence to df
df_w_quasi_p = pd.read_csv("../../../RBC/3_Persistence_tagged/2_human_to_VA/Persistence_RBC_all.csv")
#summarising all kinds of lexical quasi-persistence, i.e., writing True in new column, if lexical SPP was produced by VA (due to persistence tagging direction in this df, only the VA produced SPPs, thus no need to check speaker)
df_w_quasi_p["lexical_quasi_persistence_summarised"] = df_w_quasi_p[["persistence_unigrams_lemma", "persistence_bigrams_lemma", "persistence_trigrams_lemma", "persistence_quadrigrams_lemma"]].applymap(lambda x: str(x).startswith("SPP")).any(axis=1)
df["quasi-persistence"] = df_w_quasi_p["lexical_quasi_persistence_summarised"]
df

,id,word,lemma,pos_finegrained,pos_coarse,speaker,interaction_id,turn_id,merged,participant_id,setting,start,end,alternating_nonagentivity_nonprimed,alternating_thanking,quasi-persistence
0,1,die,der,ART.Nom.Pl.Masc,ART,Instruction,Instructions 1 - 3,Instruction,no,20170703B,00_R,Instruction,Instruction,no,no,False
1,2,anderen,anderer,ADJA.Pos.Nom.Pl.Masc,PIDAT,Instruction,Instructions 1 - 3,Instruction,no,20170703B,00_R,Instruction,Instruction,no,no,False
2,3,Freunde,Freund,NN.Nom.Pl.Masc,NN,Instruction,Instructions 1 - 3,Instruction,no,20170703B,00_R,Instruction,Instruction,no,no,False
3,4,haben,haben,VAFIN.3.Pl.Pres.Ind,VAFIN,Instruction,Instructions 1 - 3,Instruction,no,20170703B,00_R,Instruction,Instruction,no,no,False
4,5,sich,sie,PRF.3.Acc.Pl,PRF,Instruction,Instructions 1 - 3,Instruction,no,20170703B,00_R,Instruction,Instruction,no,no,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39503,39504,bis,bis,APPR,APPR,A,90,35,no,20170720J,03_S,185.708556,188.608559,no,no,False
39504,39505,Freitag,Freitag,NN.Acc.Sg.Masc,NN,A,90,35,no,20170720J,03_S,185.708556,188.608559,no,no,False
39505,39506,danke,danke,ADV,NGIRR,S,90,36,no,20170720J,03_S,188.825381,190.356691,no,danke,False
39506,39507,auf,auf,APPR,APPR,S,90,36,no,20170720J,03_S,188.825381,190.356691,no,no,False


In [35]:
reload(quantification)
variation_sample = quantification.prepare_data_for_modeling(df, alternating, beta_variants=["man", "werden"])
variation_sample.to_csv("nonagentivity_for_analysis.csv")
variation_sample

,CURRENT,PREVIOUS,PREVIOUS_SPEAKER,PREVIOUS_DISTANCE,PREVIOUS_DISTANCE_LOG,PREVIOUS_BETA_MAN,PREVIOUS_BETA_WERDEN,QUASI_PERSISTENCE,HUMAN_ID,INTERACTION_ID,TURN_LENGTH,CONFEDERATE
1741,man,man,S,4,1.386294,False,False,False,20170717A,4,15,False
4705,man,man,S,100,4.605170,False,False,False,20170717C,10,12,False
11618,man,man,S,203,5.313206,False,True,True,20170718B,26,15,False
21699,man,man,S,5,1.609438,False,False,True,20170719C,51,23,False


In [27]:
len(variation_sample) #number of variants drops from 181 to 164 due to variants being uttered first in an interaction

4

In [28]:
variation_sample.CURRENT.value_counts(normalize=True)

man    1.0
Name: CURRENT, dtype: float64